# Model analysis

In [1]:
import project_path  # Always import this first

In [2]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from utils.concrete import ConcreteSelect, decoder_1l, decoder_2l, decoder_3l
from utils.dataset import MRISelectorSubjDataset
from utils.env import DATA_PATH
from utils.logger import logger

In [3]:
ROOT_PATH = Path().cwd().parent
logger.debug(ROOT_PATH)

2021-07-01 12:09:19,251 - geometric-dl - DEBUG - C:\Users\user\Workspace\geometric-dl-dmri (<ipython-input-3-6bfbdaf6256e>:2)


In [4]:
logger.info("torch version %s", torch.__version__)

2021-07-01 12:09:19,271 - geometric-dl - INFO - torch version 1.9.0 (<ipython-input-4-3c2fef3d866b>:1)


In [42]:
# use gpu if available, else cpu
has_cuda = torch.cuda.is_available()

logger.info("Is the GPU available? %s", has_cuda)
logger.info("Current device: %s", torch.cuda.current_device())
logger.info("Device count: %s", torch.cuda.device_count())

device = torch.device("cuda" if has_cuda else "cpu")
if has_cuda:
    torch.cuda.set_device(0)
    logger.info("Using device: %s", torch.cuda.get_device_properties(device))
else:
    logger.warning("No GPU dectected! Training will be extremly slow")

2021-07-01 12:29:49,800 - geometric-dl - INFO - Is the GPU available? True (<ipython-input-42-0077c854e8eb>:4)
2021-07-01 12:29:49,800 - geometric-dl - INFO - Current device: 0 (<ipython-input-42-0077c854e8eb>:5)
2021-07-01 12:29:49,800 - geometric-dl - INFO - Device count: 1 (<ipython-input-42-0077c854e8eb>:6)
2021-07-01 12:29:49,800 - geometric-dl - INFO - Using device: _CudaDeviceProperties(name='Quadro RTX 4000', major=7, minor=5, total_memory=8192MB, multi_processor_count=36) (<ipython-input-42-0077c854e8eb>:11)


## Loading the models

First we need to cleanup the filename string. For this we define some helper functions. The string does not really have a nice structure so the cleanup functions are a bit messy.

In [43]:
def extract_features(feature_str):
    """Create a Dict with all the features from the model, extracted from its file name"""
    strdate = feature_str[:14]
    features_dict = to_dict(
        [split_str(substr) for substr in feature_str[15:].split("=")]
    )
    features_dict["datetime"] = datetime.strptime(strdate, "%Y%m%d%H%M%S")

    return features_dict


def split_str(s):
    if s.startswith("decoder"):
        # name of the decoder function string contains a '_' character so it needs special treatment
        split = s.split("_", 2)
        return ["_".join(split[:2]), split[-1]]
    return s.split("_", 1)


def to_dict(fs):
    return dict([fs[i][-1], l[0]] for i, l in enumerate(fs[1:]))

Create lookup Dict to map decoder name to decoder function

In [44]:
decoder_dict = dict(
    [decoder.__name__, decoder] for decoder in [decoder_1l, decoder_2l, decoder_3l]
)

Load the models and store them in a pandas dataframe

In [45]:
models_path = Path(ROOT_PATH, "runs", "models")

frames = []
for model_path in models_path.glob("*.pt"):
    stem = model_path.stem
    #     logger.debug('found: %s', stem)
    features_dict = extract_features(stem)
    features_dict["path"] = model_path

    # Convert types
    features_dict["lr"] = float(features_dict["lr"])
    features_dict["batch_size"] = int(features_dict["batch_size"])
    features_dict["num_epochs"] = int(features_dict["num_epochs"])
    features_dict["n_features"] = int(features_dict["n_features"])
    features_dict["test"] = int(features_dict["test"])

    df_part = pd.DataFrame(features_dict, index=[features_dict["datetime"]])
    frames.append(df_part)

df = pd.concat(frames)
df

,lr,batch_size,num_epochs,n_features,decoder,test,datetime,path
2021-06-28 10:44:35,0.001,256,2000,21,decoder_1l,15,2021-06-28 10:44:35,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-28 14:48:27,0.001,256,2000,21,decoder_2l,15,2021-06-28 14:48:27,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-28 19:31:11,0.001,256,2000,21,decoder_3l,15,2021-06-28 19:31:11,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 00:47:25,0.001,256,2000,42,decoder_1l,15,2021-06-29 00:47:25,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 04:45:53,0.001,256,2000,42,decoder_2l,15,2021-06-29 04:45:53,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 09:33:59,0.001,256,2000,42,decoder_3l,15,2021-06-29 09:33:59,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 14:51:26,0.001,256,2000,84,decoder_1l,15,2021-06-29 14:51:26,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 18:54:22,0.001,256,2000,84,decoder_2l,15,2021-06-29 18:54:22,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-29 23:41:58,0.001,256,2000,84,decoder_3l,15,2021-06-29 23:41:58,C:\Users\user\Workspace\geometric-dl-dmri\runs...
2021-06-30 04:53:13,0.001,256,2000,168,decoder_1l,15,2021-06-30 04:53:13,C:\Users\user\Workspace\geometric-dl-dmri\runs...


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15 entries, 2021-06-28 10:44:35 to 2021-07-01 05:30:27
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   lr          15 non-null     float64       
 1   batch_size  15 non-null     int64         
 2   num_epochs  15 non-null     int64         
 3   n_features  15 non-null     int64         
 4   decoder     15 non-null     object        
 5   test        15 non-null     int64         
 6   datetime    15 non-null     datetime64[ns]
 7   path        15 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 1.1+ KB


## Model evalutation

In [47]:
test_set = MRISelectorSubjDataset(
    Path(ROOT_PATH, "data"), "data_.hdf5", "header_.csv", np.array([15])
)
test_gen = DataLoader(
    test_set,
    batch_size=256//2,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    drop_last=True,
)

In [48]:
mse_loss = torch.nn.MSELoss(reduction="none").to(device)

for index, row in df.iterrows():
    model_path = row["path"]
    model = ConcreteSelect(
        output_dim=int(row["n_features"]),
        input_shape=1344,
        decoder=decoder_dict[row["decoder"]],
        device=device,
    ).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for target in test_set:
            pred, _ = model(torch.from_numpy(target).to(device))
            total_loss += mse_loss(target, pred)

    logger.info("loss %s", total_loss / len(test_gen))

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [49]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.